<a href="https://colab.research.google.com/github/ankitrahejagatech/crewaimultiagentdemo/blob/main/L3_customer_support.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [23]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [13]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [19]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/utils.py')

Mounted at /content/drive


- Import libraries, API and LLM

In [20]:
from crewai import Agent, Task, Crew

In [24]:
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = "<>"

## Role Playing, Focus and Cooperation

In [26]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task.

In [27]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [28]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [29]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [31]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [32]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [33]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [35]:
inputs = {
    "customer": "PM Excercises",
    "person": "Bijan",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

- Display the final result as Markdown.

In [36]:
from IPython.display import Markdown
Markdown(result)

Dear Bijan,

Thank you for reaching out with your query about setting up a Crew and adding memory to it. I'm here to provide you with comprehensive guidance on this matter.

To set up a Crew and add memory to it, you can follow the steps outlined in the CrewAI documentation, specifically in the "Assembling and Activating Your CrewAI Team" section. Here is a summary of the steps you need to take:

1. Installation: Install CrewAI and any necessary packages for your project. Ensure compatibility with Python >=3.10, <=3.13.
   - Command: 
     ```
     pip install crewai
     pip install 'crewai[tools]'
     ```

2. Assemble Your Agents: Define your agents with distinct roles, backstories, and capabilities like memory usage.
   - Example code snippet:
     ```
     from crewai import Agent
     # Creating a senior researcher agent with memory and verbose mode
     researcher = Agent(
         role='Senior Researcher',
         goal='Uncover groundbreaking technologies in {topic}',
         verbose=True,
         memory=True,
         backstory="Driven by curiosity, you're at the forefront of innovation...",
         tools=[search_tool],
         allow_delegation=True
     )
     ```

3. Define the Tasks: Detail specific objectives for your agents with memory and output customization.
   - Example code snippet:
     ```
     from crewai import Task
     # Research task
     research_task = Task(
         description="Identify the next big trend in {topic}...",
         expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
         tools=[search_tool],
         agent=researcher,
     )
     ```

4. Form the Crew: Combine your agents into a crew with memory and enhanced configurations for task execution.
   - Example code snippet:
     ```
     from crewai import Crew, Process
     # Forming the tech-focused crew with enhanced configurations
     crew = Crew(
         agents=[researcher, writer],
         tasks=[research_task, write_task],
         process=Process.sequential,
         memory=True,
         cache=True,
         max_rpm=100,
         share_crew=True
     )
     ```

5. Kick It Off: Initiate the task execution process with your enhanced crew ready for collaboration.
   - Example command:
     ```
     result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
     print(result)
     ```

Utilizing Memory Capabilities in Crew Task Execution and Collaboration:

Memory in the Crew platform is crucial for optimizing task execution and collaboration. Here's how memory capabilities are utilized:
1. Task Execution: Memory stores task-related data for quick access and efficient execution. It also caches data for faster processing.
2. Collaboration: Memory facilitates real-time collaboration, storing and updating task information for seamless team interaction.

By following these steps and understanding how memory is used in task execution and collaboration, you can successfully set up a Crew with memory capabilities. If you need further assistance or clarification on any step, feel free to reach out to us.

Best regards,

[Your Name]
Senior Support Representative
crewAI